In [1]:
import numpy as np
import matplotlib.pylab as plt
import os
import random

import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
device = 'cuda:0' if torch.cuda.is_available() else "cpu"

### Generating Synthetic Data

In [3]:
# generate N vectors of 32 bytes
# generate N label vectors of 31 zero bytes and 1 non-zero byte at target_idx
def gen_data(N=15000, d=32, low=0, high=127, target_idx=1):
    data = np.random.randint(low=low, high=high, size=(N,d))
    labels = np.zeros((N, d), dtype=int)
    labels[:, target_idx] = data[:, target_idx]
    return data, labels

In [4]:
data = gen_data()
features = data[0]
print("Feature Vectors:")
print(features)
labels = data[1]
print("Labels:")
print(labels)
test_data = gen_data(N=3000)
features_test = test_data[0]
print("Test Feature Vectors:")
print(features_test)
labels_test = test_data[1]
print("Test Labels:")
print(labels_test)

Feature Vectors:
[[116 106  51 ... 119 124   1]
 [104 103 120 ...  39  71   2]
 [ 56  45  64 ... 119  17  91]
 ...
 [ 73  93  29 ... 115  58   5]
 [ 29 126  86 ...  88  16  65]
 [ 60 118  31 ...  44  87 109]]
Labels:
[[  0 106   0 ...   0   0   0]
 [  0 103   0 ...   0   0   0]
 [  0  45   0 ...   0   0   0]
 ...
 [  0  93   0 ...   0   0   0]
 [  0 126   0 ...   0   0   0]
 [  0 118   0 ...   0   0   0]]
Test Feature Vectors:
[[ 99  51  84 ...  19  87   5]
 [ 80  81  83 ...  14  17  83]
 [ 59  64  42 ...  14  91 126]
 ...
 [121  50 119 ...  73  82  79]
 [ 99  73 102 ...  70  81  86]
 [ 31  31  32 ...  27  59   1]]
Test Labels:
[[ 0 51  0 ...  0  0  0]
 [ 0 81  0 ...  0  0  0]
 [ 0 64  0 ...  0  0  0]
 ...
 [ 0 50  0 ...  0  0  0]
 [ 0 73  0 ...  0  0  0]
 [ 0 31  0 ...  0  0  0]]


### Defining Zero-Hidden Layer NN = Linear Regression

In [5]:
net = nn.Sequential(
    nn.Linear(features.shape[1], features.shape[1])
)

criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=1e-3)

#### Generating Synthetic Data

In [6]:
data = gen_data(N=15000)
features = data[0]
#print("Feature Vectors:")
#print(features)
labels = data[1]
#print("Labels:")
#print(labels)
test_data = gen_data(N=5000)
features_test = test_data[0]
#print("Test Feature Vectors:")
#print(features_test)
labels_test = test_data[1]
#print("Test Labels:")
#print(labels_test)

In [7]:
def validate(net, test_data, test_target, net_type=None): 
    test_data_torch = torch.from_numpy(test_data).float()
    test_target_torch = torch.from_numpy(test_target).float()
    # prediction
    test_pred = np.rint(net(test_data_torch).detach().numpy())

    #print(np.concatenate((pred, test_target_torch.detach().numpy()), axis=1))
    i = random.randint(0, test_data.shape[0] - 1)
    print("Pred[", i, "]:")
    print(test_pred[i])
    print("Target[", i, "]:")
    print(test_target_torch.detach().numpy()[i])
    print("Input[", i, "]:")
    print(test_data_torch.detach().numpy()[i])

    test_accuracy = (test_target_torch.detach().numpy() == test_pred).mean()
    test_rmse = (np.sqrt((test_pred-test_target_torch.detach().numpy())**2).mean())

    return test_accuracy, test_rmse


def train_net(net, train_data, train_target, test_data, test_target, N_epochs=None, print_freq=100, net_type=None):
    
    net = net.to(device)
    train_data_torch = torch.from_numpy(train_data).float().to(device)
    train_target_torch = torch.from_numpy(train_target).float().to(device)
    
    if net_type=='conv':
        train_data_torch = train_data_torch.unsqueeze(1).unsqueeze(2)

    net = net.train()
    for i in range(N_epochs):
        pred = net(train_data_torch)
        loss = criterion(pred, train_target_torch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % print_freq == 0:
            train_accuracy, train_rmse = validate(net, train_data, train_target, net_type=net_type)
            test_accuracy, test_rmse = validate(net, test_data, test_target, net_type=net_type)
            net = net.train()
            print(f'Epoch = {i} loss={loss} train_acc = {train_accuracy:.3f} test_acc = {test_accuracy:.3f} train_rmse = {train_rmse:.3f}  test_rmse = {test_rmse:.3f}')
            
    return net


In [8]:
net = train_net(net, features, labels, features_test, labels_test, N_epochs=5000, print_freq=10)

torch.save(net, "nn_regression_0")

Pred[ 10833 ]:
[ -80. -107.  -31.   60.  -66.   29.   56.   32.    8.    6.  -17.   21.
   10.  -77.    3.   11.   27.  -23.  -28.  -36.   24.    2.   36.   25.
    0.   40.   39.   44.  -46.  -73.   23.   54.]
Target[ 10833 ]:
[ 0. 16.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Input[ 10833 ]:
[ 18.  16. 124.  49. 107. 100.  69. 124.  97. 100. 123.  34. 125. 122.
  58. 101.  75.  26.  41.  48.  15.  79.  59.  62.  19. 126.  17.  30.
  23.  71. 101.  66.]
Pred[ 388 ]:
[ -62. -106.  -49.   37.  -68.   11.   81.   29.  -29.  -62.   18.   49.
   -8.  -45.   -8.  -15.   15.   17.   -5.  -47.  -13.   -4.   33.   25.
   16.   24.   29.  -13.  -35.  -35.   57.   61.]
Target[ 388 ]:
[ 0. 17.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Input[ 388 ]:
[122.  17.  97.  63.  90.  92.  40.  33.  11.  11.  62.  81.  87. 104.
  49. 115.  52.   7. 

Pred[ 6746 ]:
[-24.  57.  -4.  29.   2.   3.   2. -18. -13.  -1.   3.   7.  12. -24.
 -11.  -5.  17.  17.  -0.  -5.  10.   5.   1.  19.  17. -13.  -5.  -0.
  -4.   2.   2.  -1.]
Target[ 6746 ]:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]
Input[ 6746 ]:
[102.   0.  32.  86.   5.  35.  38.  75.  96.  88.  28.  74.  42.   4.
   3.  45.  56.  10.  28. 103.  89.  54.  26.  27.  25. 108.  59.  29.
  99.  60.  82.  41.]
Pred[ 693 ]:
[-12.   4.   2.   7.  -0. -11.  10.   5.   2.   1.  12.  25.  22.  -5.
 -10.   6. -36.  -3.   4.  17. -14.  15.  -4.  20.  -5.   5.   8. -15.
  23. -23.  15.  -2.]
Target[ 693 ]:
[ 0. 43.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Input[ 693 ]:
[ 30.  43. 119. 123.  89.  43.  22.  30.  24.  15. 120.  80.   1.  75.
  19.  39.  35.  10.  64.  68.  23.  25. 114.  71.  62.  23. 102. 112.
  93.  83.  72.  45.]
Epoch = 90 loss=330.1166076

 -14.  11. -12. -19.]
Target[ 3531 ]:
[ 0. 75.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Input[ 3531 ]:
[ 23.  75.  24. 108.  40. 107.  98.  93.  49. 109.  11.   0.  23. 123.
  55. 116.   1.  77.  43.  85.  69.  55.  40.  15.  56.  30.   3.  29.
  54. 119.  73. 102.]
Epoch = 160 loss=214.30186462402344 train_acc = 0.041 test_acc = 0.041 train_rmse = 10.217  test_rmse = 10.234
Pred[ 2201 ]:
[ 16.  -2.   4.  -5. -17.   1. -28.  15.  -2.  -7.  -2.  -1.  -0.  20.
   3. -26. -15.  -6.  15.   1.   4.   7. -22.   7.  -5.  -5. -28.  -8.
  -4.  -5.  -0.  -1.]
Target[ 2201 ]:
[ 0. 78.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Input[ 2201 ]:
[114.  78.  99.  22. 100.  11.  61.  26.   0.  19.  76.  56.  40. 109.
   3.  60.  77.  99. 113.  20.  61. 112.  88. 105.  15.  41.  23.  15.
  39.  79.  96.  55.]
Pred[ 2766 ]:
[  7.  15.  -2.  12. -1

Pred[ 10252 ]:
[  5.  34.   7.   3.  12.  -2.  11.   6.  -1.   1.  -1.   9.  -2.   8.
   1.  -7.  -6.   4.  -3.  -7.  -1.  17.   5.   3.  -3.  22.   4.  -1.
   4.  16.  -2. -10.]
Target[ 10252 ]:
[ 0. 93.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Input[ 10252 ]:
[  6.  93.  38.  54.  66.  33.  50.  89.  41.   1. 125.   2.   8.  69.
  81.  20.  74.  84.  66.  91.  92.  53.  98.  23.  73.  27.   4.  23.
  56.  54.  82.  15.]
Pred[ 2642 ]:
[ -8.  55. -20.   8.  -2.   5.  -7. -18.   0.  -4.   1.  -4.  -0.  -6.
  -1. -17.   1.  -6.   4. -12.   1.   1.   1.   2.   2.   8.   5.   0.
  -0.  -5.   0.   3.]
Target[ 2642 ]:
[ 0. 40.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Input[ 2642 ]:
[ 28.  40. 116.  14.  88.  12. 104.  46.  71.  49.   3.  90.  31.  97.
  74. 106.  20.  54.  47. 109.  73.  76.  15.  74.  85. 111.  32.  72.
  46.  46. 

Pred[ 608 ]:
[-25.  69.   1.  -7. -37.  -2.  -6.  -3.   1.   1.   0.  -2.  -0.  -1.
   0.  -3.  -2.  -1.   3. -11.   0.   2.   1.   1.  -2.  -2.  -2.  -0.
   3. -16.  -1.  -7.]
Target[ 608 ]:
[ 0. 12.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Input[ 608 ]:
[ 73.  12.  14.  39.  13.  35. 121.  11. 115.  65. 110.  74.  13. 106.
  62.  20.  11. 103.  34.  41.  98. 114.  21.  78.  68.  43. 101.  80.
  54.  32.  43. 120.]
Pred[ 3535 ]:
[ 14.  23.  -7. -12. -25.  -2.  -2.   6.   2.   1.  -1.   6.   1.  -1.
   1.  11.  -5.  -2.  -4.   2.   0.  -0.   2.  -1.   0.  12.  -6.  -0.
  -7.  -2.   1.   5.]
Target[ 3535 ]:
[ 0. 55.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Input[ 3535 ]:
[ 19.  55.  19.   6.  78.  16.  40.   9.   7.  39. 116.  87.  84.  75.
  69.  84. 105. 124.  30. 100.  14.  72. 109.  51.  31.   3. 100. 126.
  42.  52.   3.  

...

Pred[ 1156 ]:
[-0. 89. -0. -0.  0.  0.  0. -0.  0.  0. -0. -0.  0. -0.  0.  0. -0. -0.
  0.  0.  0.  0. -0.  0.  0.  0. -0. -0. -0.  0.  0. -0.]
Target[ 1156 ]:
[ 0. 89.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Input[ 1156 ]:
[ 71.  89.  12.  59.  20.   1.  19. 126. 106.  97.  59.  40. 108. 125.
 104.  55.  79.   5.  59.  12.  28. 100.  84.  23.   6. 114.  54.  21.
  53.  19.  72.  48.]
Epoch = 4610 loss=0.002373238792642951 train_acc = 1.000 test_acc = 1.000 train_rmse = 0.000  test_rmse = 0.000
Pred[ 6665 ]:
[ 0. 55.  0.  0. -0. -0. -0.  0. -0. -0.  0.  0. -0.  0. -0. -0.  0.  0.
 -0. -0.  0. -0.  0. -0. -0. -0.  0.  0.  0. -0. -0.  0.]
Target[ 6665 ]:
[ 0. 55.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Input[ 6665 ]:
[ 44.  55.  80.  49.  99.  80.  16.  74.  36.  76.  96.  68. 102.  88.
 110.  91.  91.  78. 112. 106.  92.  24

Pred[ 12700 ]:
[ 0. 88.  0.  0. -0. -0. -0.  0. -0. -0.  0.  0. -0.  0. -0. -0.  0.  0.
 -0. -0. -0. -0.  0. -0. -0. -0.  0.  0.  0. -0. -0.  0.]
Target[ 12700 ]:
[ 0. 88.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Input[ 12700 ]:
[ 47.  88.  48.  25.  90.  77. 112.  37.  61. 119.  45. 120.   1.   8.
  79.  74. 103.   9.  54. 126.  44.  63.  54.  97. 124.  81.  52. 111.
  29. 106.  37.  43.]
Pred[ 1247 ]:
[-0. 23. -0. -0.  0.  0.  0. -0.  0.  0. -0. -0.  0. -0.  0.  0. -0. -0.
  0.  0.  0.  0. -0.  0.  0.  0. -0. -0. -0.  0.  0. -0.]
Target[ 1247 ]:
[ 0. 23.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Input[ 1247 ]:
[ 77.  23.  38.  13.  94. 121. 125.  88.  78.   8.  36.  17.  57.  36.
  73.  56.  28.  46.  37.  32.  89.  49.  89.  92.  96.   2.  71.  34.
  85. 105.  82.   7.]
Epoch = 4700 loss=8.812584565021098e-05 train_acc = 1.0

Pred[ 6181 ]:
[-0. 54. -0. -0.  0.  0.  0. -0.  0.  0. -0. -0.  0. -0.  0.  0. -0. -0.
  0.  0.  0.  0. -0.  0.  0.  0. -0. -0. -0.  0. -0. -0.]
Target[ 6181 ]:
[ 0. 54.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Input[ 6181 ]:
[ 53.  54.   2.  30.  74.  20.  17.   8.  38.  96.  46.  42. 117.  98.
   3. 121.  44.  78.   3.   7.  57.  68.  40.  35.  73.  63. 105.  91.
  72.  41.  83.  69.]
Pred[ 3653 ]:
[ 0. 95.  0.  0. -0. -0. -0.  0. -0. -0.  0.  0. -0.  0. -0. -0.  0.  0.
 -0. -0. -0. -0.  0. -0. -0. -0.  0.  0.  0. -0. -0.  0.]
Target[ 3653 ]:
[ 0. 95.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Input[ 3653 ]:
[ 90.  95.  82. 102.   8.  22. 113.  88.  79.  82.  43.  11.  58. 116.
  82.  86. 121.  20.  72.  65.  60.  44.  29. 125.  53. 109.  67.   6.
  58.  78.  18.  84.]
Epoch = 4780 loss=0.003394083585590124 train_acc = 1.000 t

Pred[ 9410 ]:
[  0. 105.   0.   0.  -0.  -0.  -0.   0.  -0.  -0.   0.   0.  -0.   0.
  -0.  -0.   0.   0.  -0.  -0.  -0.  -0.   0.  -0.  -0.  -0.   0.   0.
   0.  -0.   0.   0.]
Target[ 9410 ]:
[  0. 105.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.]
Input[ 9410 ]:
[ 34. 105.  71.  82.   6.  44.   7. 110.  62.  25. 122. 125.  48.   6.
  26. 113.  20. 112.  30.   1. 115. 123.  69. 121.  46. 109.  75. 122.
  33. 100.  30.  34.]
Pred[ 3143 ]:
[ -0. 126.  -0.  -0.   0.   0.   0.  -0.   0.   0.  -0.  -0.   0.  -0.
   0.   0.  -0.  -0.   0.   0.   0.   0.  -0.   0.   0.   0.  -0.   0.
  -0.   0.   0.  -0.]
Target[ 3143 ]:
[  0. 126.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.]
Input[ 3143 ]:
[ 53. 126.  55.  99.  14.   6.  99.  20. 106. 118.  50.  15.  93.  50.
 124.  14.  86. 107

Pred[ 2692 ]:
[ 0.  8.  0.  0. -0.  0. -0.  0. -0. -0.  0.  0. -0.  0. -0. -0.  0.  0.
 -0. -0. -0. -0.  0. -0. -0. -0.  0.  0.  0. -0. -0.  0.]
Target[ 2692 ]:
[0. 8. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]
Input[ 2692 ]:
[ 95.   8.  93. 125. 109.  51. 122. 103. 105.  54. 121.  37.   1.  35.
   9.  42. 108.  65.  90.  68.  64.  97.  80.  64.  15.  24.  20.  71.
 112.  31.  76.  40.]
Epoch = 4980 loss=0.0002127628104062751 train_acc = 1.000 test_acc = 1.000 train_rmse = 0.000  test_rmse = 0.000
Pred[ 13300 ]:
[ 0. 85.  0.  0. -0.  0. -0.  0. -0. -0.  0.  0. -0.  0. -0. -0.  0.  0.
 -0. -0. -0. -0.  0. -0. -0. -0.  0.  0.  0. -0. -0.  0.]
Target[ 13300 ]:
[ 0. 85.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Input[ 13300 ]:
[ 26.  85.  55.  15.   4.  10.  61. 100.  46.  90.  94.  89. 109.  82.
 117.  89.  90.  49.  73.  47.  73.  46. 105.  65.  95.  58. 126.  

In [9]:
for w in net.parameters():
    print(w)
    print(w.shape)


Parameter containing:
tensor([[ 2.2592e-05,  2.1619e-05,  2.1356e-05,  ...,  2.1233e-05,
          2.1597e-05,  2.3779e-05],
        [-1.0109e-04,  9.9990e-01, -9.5541e-05,  ..., -9.4970e-05,
         -9.6600e-05, -1.0643e-04],
        [ 4.8394e-05,  4.6317e-05,  4.5760e-05,  ...,  4.5470e-05,
          4.6254e-05,  5.0927e-05],
        ...,
        [-1.6812e-06, -7.0523e-06, -1.4225e-06,  ..., -1.1893e-07,
         -1.8283e-07, -2.9122e-06],
        [ 1.2449e-05,  1.1914e-05,  1.1769e-05,  ...,  1.1701e-05,
          1.1901e-05,  1.3103e-05],
        [ 3.0878e-05,  2.9554e-05,  2.9188e-05,  ...,  2.9018e-05,
          2.9517e-05,  3.2505e-05]], requires_grad=True)
torch.Size([32, 32])
Parameter containing:
tensor([-0.0447,  0.2000, -0.0957,  0.0240, -0.0773, -0.0199,  0.0627,  0.0744,
         0.0457,  0.1583,  0.0324,  0.0456,  0.0242, -0.0162,  0.0442, -0.0503,
        -0.0078,  0.0030,  0.0220,  0.0924,  0.1195,  0.0895, -0.1651,  0.0942,
        -0.0223, -0.1371,  0.1270,  0.0809,

### Defining NN

In [101]:
net = nn.Sequential(
    nn.Linear(features.shape[1], 2**8),
    nn.ReLU(), #rectified linear unit
    nn.Linear(2**8, 2**12),
    nn.ReLU(),
    #nn.Linear(2**12, 2**10),
    #nn.ReLU(),
    nn.Linear(2**12, features.shape[1]),
    #nn.Sigmoid()
)

criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=1e-3)

In [24]:
def validate(net, test_data, test_target, net_type=None): 
    test_data_torch = torch.from_numpy(test_data).float()
    test_target_torch = torch.from_numpy(test_target).float()
    # prediction
    test_pred = np.rint(net(test_data_torch).detach().numpy())

    #print(np.concatenate((pred, test_target_torch.detach().numpy()), axis=1))
    i = random.randint(0, test_data.shape[0] - 1)
    print("Pred[", i, "]:")
    print(test_pred[i])
    print("Target[", i, "]:")
    print(test_target_torch.detach().numpy()[i])
    print("Input[", i, "]:")
    print(test_data_torch.detach().numpy()[i])

    test_accuracy = (test_target_torch.detach().numpy() == test_pred).mean()
    test_rmse = (np.sqrt((test_pred-test_target_torch.detach().numpy())**2).mean())

    return test_accuracy, test_rmse


def train_net(net, train_data, train_target, test_data, test_target, N_epochs=None, print_freq=100, net_type=None):

#     print("Train Vectors:")
#     print(train_data)
#     print("Train Labels:")
#     print(train_target)
#     print("Test Vectors:")
#     print(test_data)
#     print("Test Labels:")
#     print(test_target)
    
    net = net.to(device)
    train_data_torch = torch.from_numpy(train_data).float().to(device)
    train_target_torch = torch.from_numpy(train_target).float().to(device)
    
    print("train_data_torch")
    print(train_data_torch)
    print("train_target_torch")
    print(train_target_torch)
    
    if net_type=='conv':
        train_data_torch = train_data_torch.unsqueeze(1).unsqueeze(2)

    net = net.train()
    for i in range(N_epochs):
        #pred = net(train_data_torch.float())
        #loss = criterion(pred, train_target_torch.float())
        pred = net(train_data_torch)
        loss = criterion(pred, train_target_torch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % print_freq == 0:
            train_accuracy, train_rmse = validate(net, train_data, train_target, net_type=net_type)
            test_accuracy, test_rmse = validate(net, test_data, test_target, net_type=net_type)
            net = net.train()
            print(f'Epoch = {i} loss={loss} train_acc = {train_accuracy:.3f} test_acc = {test_accuracy:.3f} train_rmse = {train_rmse:.3f}  test_rmse = {test_rmse:.3f}')
            
    return net

        
        

In [103]:
net = train_net(net, features, labels, features_test, labels_test, N_epochs=5000, print_freq=10)

torch.save(net, "nn_mask_0")

train_data_torch
tensor([[  1.,  90.,  67.,  ...,  78.,  60., 101.],
        [ 55.,  45., 100.,  ...,  31., 104.,  40.],
        [ 23., 110.,  98.,  ..., 104.,  30.,  94.],
        ...,
        [ 23.,  28.,  54.,  ...,  25.,  76.,  10.],
        [ 90.,  77.,  55.,  ...,   6.,  55., 126.],
        [ 23.,  22.,  27.,  ..., 102., 101.,  68.]])
train_target_torch
tensor([[  0.,  90.,   0.,  ...,   0.,   0.,   0.],
        [  0.,  45.,   0.,  ...,   0.,   0.,   0.],
        [  0., 110.,   0.,  ...,   0.,   0.,   0.],
        ...,
        [  0.,  28.,   0.,  ...,   0.,   0.,   0.],
        [  0.,  77.,   0.,  ...,   0.,   0.,   0.],
        [  0.,  22.,   0.,  ...,   0.,   0.,   0.]])
Pred[ 2506 ]:
[-3278.  9901. -4018.  3724.  4017.  1536.  1220.  4078.  3505.  3859.
 -4083.  2248. -4007. -3004.  3163.  4245.  3285. -3648.  3711. -3886.
 -3556.   631.  3237.  4200.  3705. -3694. -3661. -3846. -4569. -3320.
 -3878.  3694.]
Target[ 2506 ]:
[0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

Pred[ 1846 ]:
[-0. 48. -0. -0.  0.  0.  0. -0.  0. -0.  0.  0. -0. -0.  0.  0. -0. -0.
 -0.  0.  0.  0. -0. -0. -0.  0. -0.  0.  0.  0.  0.  0.]
Target[ 1846 ]:
[ 0. 82.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Input[ 1846 ]:
[ 60.  82. 125.  46. 119. 104.  95.  61.   1. 111.  44.  59.  18. 115.
  30.  15.  56.  95. 106. 105.   7.  84.  71.  30.  60.  27.   7. 108.
  85. 123. 112. 114.]
Pred[ 277 ]:
[-0. 69.  0. -0.  0.  0.  0. -0.  0. -0.  0.  0.  0.  0.  0. -0. -0.  0.
 -0.  0.  0.  0. -0. -0. -0.  0.  0.  0.  0.  0.  0.  0.]
Target[ 277 ]:
[ 0. 87.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Input[ 277 ]:
[ 86.  87.  37.   6.  24.  82.  18.  17. 120. 107.  83.   7.   9.  27.
  63.  68. 109.  60. 102.  34.  34.  85.  80.  82.  92.   4.  25.  88.
  22.  26. 119.  32.]
Epoch = 70 loss=37.601356506347656 train_acc = 0.926 test_acc

KeyboardInterrupt: 

### More Layers, Smaller Dataset

In [118]:
data = gen_data(N=10000)
features = data[0]
print("Feature Vectors:")
print(features)
labels = data[1]
print("Labels:")
print(labels)
test_data = gen_data(N=2000)
features_test = test_data[0]
print("Test Feature Vectors:")
print(features_test)
labels_test = test_data[1]
print("Test Labels:")
print(labels_test)

Feature Vectors:
[[ 35 118  73 ... 119  25  65]
 [ 27 101 121 ... 109  26  47]
 [ 76  77  45 ...  90 124  57]
 ...
 [  0  44  88 ...   8  89  42]
 [125  21  36 ...  99  78   1]
 [ 29  12 105 ...  76   7  77]]
Labels:
[[  0 118   0 ...   0   0   0]
 [  0 101   0 ...   0   0   0]
 [  0  77   0 ...   0   0   0]
 ...
 [  0  44   0 ...   0   0   0]
 [  0  21   0 ...   0   0   0]
 [  0  12   0 ...   0   0   0]]
Test Feature Vectors:
[[104  79 117 ...   7 121  72]
 [ 47  52  48 ...  28  13  83]
 [ 11  95 112 ... 105   9 104]
 ...
 [ 79  23  50 ...  95  79  82]
 [ 45  89 108 ...  97 123  10]
 [ 65  28  94 ...  42  94  60]]
Test Labels:
[[ 0 79  0 ...  0  0  0]
 [ 0 52  0 ...  0  0  0]
 [ 0 95  0 ...  0  0  0]
 ...
 [ 0 23  0 ...  0  0  0]
 [ 0 89  0 ...  0  0  0]
 [ 0 28  0 ...  0  0  0]]


In [127]:
net = nn.Sequential(
    nn.Linear(features.shape[1], 2**12),
    nn.ReLU(), #rectified linear unit
    nn.Linear(2**12, 2**10),
    nn.ReLU(),
    nn.Linear(2**10, 2**8),
    nn.ReLU(),   
    nn.Linear(2**8, 2**8),
    nn.ReLU(),
    nn.Linear(2**8, features.shape[1]),
    #nn.Sigmoid()
)

criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=1e-3)

In [128]:
net = train_net(net, features, labels, features_test, labels_test, N_epochs=5000, print_freq=10)

torch.save(net, "nn_mask_1")

train_data_torch
tensor([[ 35., 118.,  73.,  ..., 119.,  25.,  65.],
        [ 27., 101., 121.,  ..., 109.,  26.,  47.],
        [ 76.,  77.,  45.,  ...,  90., 124.,  57.],
        ...,
        [  0.,  44.,  88.,  ...,   8.,  89.,  42.],
        [125.,  21.,  36.,  ...,  99.,  78.,   1.],
        [ 29.,  12., 105.,  ...,  76.,   7.,  77.]])
train_target_torch
tensor([[  0., 118.,   0.,  ...,   0.,   0.,   0.],
        [  0., 101.,   0.,  ...,   0.,   0.,   0.],
        [  0.,  77.,   0.,  ...,   0.,   0.,   0.],
        ...,
        [  0.,  44.,   0.,  ...,   0.,   0.,   0.],
        [  0.,  21.,   0.,  ...,   0.,   0.,   0.],
        [  0.,  12.,   0.,  ...,   0.,   0.,   0.]])
Pred[ 4730 ]:
[  0.  67.  16.   2.   3. -19.   5.  14.   3.   8.  12.   2. -28.  10.
 -15.  -0.   5. -10.  -3. -13. -10.  -8.  -3. -22.  -0. -14.   3.  -8.
  16.  -5.  19.  19.]
Target[ 4730 ]:
[ 0. 48.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 

Pred[ 4833 ]:
[ -0. 104.   0.   0.  -0.  -0.   0.   0.   0.   0.   0.  -0.  -0.   0.
   0.   0.   0.   0.   0.  -0.  -0.  -0.  -0.  -0.   0.  -0.  -0.  -0.
  -0.  -0.   0.   0.]
Target[ 4833 ]:
[  0. 102.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.]
Input[ 4833 ]:
[102. 102.  35.  10.  73. 119.  93.  12. 118.  46.  91.  48.  62. 120.
  79.  68.  44.  75.  78.   5.   7.  77.   3.  88.  20.   2. 125.  12.
  44.  23. 106.  34.]
Pred[ 475 ]:
[ 0. 50.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -0. -0. -0. -0.  0. -0.
  0.  0. -0.  0. -0. -0. -0.  0.  0. -0. -0.  0.  0. -0.]
Target[ 475 ]:
[ 0. 51.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Input[ 475 ]:
[ 48.  51. 113.  91. 126.  47.  24.  85.  64.   2.  21.  37.  31.  41.
   9.  35.  52. 115.  44.  63. 114.  61.  29.  50.  66.  24.  90.  94.
  19. 104. 115.  

Pred[ 9612 ]:
[-0. 85.  0.  0. -0. -0.  0. -0.  0. -0. -0.  0. -0.  0.  0.  0. -0. -0.
 -0. -0. -0. -0.  0.  0.  0. -0. -0.  0.  0. -0.  0.  0.]
Target[ 9612 ]:
[ 0. 86.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Input[ 9612 ]:
[119.  86.  87. 109. 116.  23.  32.  66.  42.  74.  96.  57. 108.  53.
 108.  70.  74.  33.  39.  85.  11.  50.   7.  48.  14.  71.  84.  54.
  94.  64.  98. 105.]
Pred[ 917 ]:
[ 0. 80. -0. -0.  0. -0.  0. -0.  0.  0.  0.  0.  0.  0.  0.  0. -0.  0.
  0. -0. -0. -0.  0.  0.  0.  0. -0. -0. -0. -0.  0.  0.]
Target[ 917 ]:
[ 0. 81.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Input[ 917 ]:
[ 38.  81.  39. 119.   3.  72.  35.  88. 106.  89.  27. 103.  86.  23.
   3.  18.  54. 110.  16.   2.  31. 113.   8.  74.  24.  53. 111.  73.
 125.  43.  84.  26.]
Epoch = 160 loss=0.026897598057985306 train_acc = 0.975 test_

Pred[ 4877 ]:
[ 0. 43. -0. -0.  0. -0. -0.  0. -0. -0.  0.  0. -0. -0.  0. -0. -0. -0.
 -0.  0. -0. -0.  0. -0.  0. -0. -0.  0.  0.  0.  0. -0.]
Target[ 4877 ]:
[ 0. 43.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Input[ 4877 ]:
[ 72.  43. 104. 126. 105.  66.  58.  91.  20.  54.  77. 105.  42.  52.
  89.  84.  78.  38.  49.  89.  38. 111.  86.  94.  99.  68.  49.  96.
 115.  97.  55.   3.]
Pred[ 1079 ]:
[ -0. 117.   0.   0.   0.   0.  -0.   0.  -0.  -0.  -0.  -0.   0.   0.
  -0.  -0.   0.   0.   0.   0.   0.   0.  -0.  -0.  -0.   0.   0.   0.
   0.   0.   0.   0.]
Target[ 1079 ]:
[  0. 117.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.]
Input[ 1079 ]:
[104. 117.  99. 102.  54.  86.  43.  75.  65.  99.  95. 123.  42.  33.
   6. 116.  59.  45.  98.  83.  44.  30. 118.  16.  41.  53. 101. 107.
  29.  19.  39

Pred[ 1566 ]:
[ -0. 112.   0.   0.  -0.  -0.  -0.   0.   0.   0.  -0.  -0.   0.   0.
  -0.  -0.   0.  -0.  -0.   0.  -0.  -0.  -0.   0.  -0.   0.   0.  -0.
   0.  -0.  -0.  -0.]
Target[ 1566 ]:
[  0. 112.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.]
Input[ 1566 ]:
[  5. 112.  53.  68.  75.   9.  43.  99.  16.  92.  75. 113.  78. 119.
   4.   9.  49.   1.  76.  19. 104.  55.  11.  52. 106.  73.  93.  69.
   7. 124. 117.  63.]
Pred[ 1849 ]:
[ 0. 27. -0. -0.  0. -0.  0. -0.  0.  0. -0. -0.  0. -0. -0. -0.  0. -0.
 -0. -0. -0. -0.  0. -0. -0. -0. -0. -0.  0. -0. -0. -0.]
Target[ 1849 ]:
[ 0. 28.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Input[ 1849 ]:
[121.  28.  16.   7. 106.  63. 122.  66.  39.  76.  89. 111.  69.  23.
  59. 117.  27.  68.  94.   3.  34.  66.  64.  25.  72.  53.  26.  83.
  87.  77.  41

Pred[ 2086 ]:
[ 0. 98.  0. -0. -0.  0.  0. -0.  0.  0. -0.  0.  0.  0.  0.  0. -0.  0.
  0.  0. -0.  0.  0.  0.  0. -0. -0. -0.  0.  0.  0.  0.]
Target[ 2086 ]:
[ 0. 99.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Input[ 2086 ]:
[125.  99. 122.  53.  88.  84. 103. 107.  54. 104. 112.  18.  67.  41.
  90.  55.  70. 115.  20.  92.  71.  38.  55.  35.  38.  59.  20.   8.
  80. 109.  81. 124.]
Pred[ 1277 ]:
[ 0. 56.  0. -0.  0.  0.  0.  0.  0. -0.  0. -0.  0.  0.  0.  0. -0.  0.
 -0. -0. -0.  0.  0.  0.  0. -0. -0.  0. -0.  0. -0. -0.]
Target[ 1277 ]:
[ 0. 57.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Input[ 1277 ]:
[ 78.  57.  41.  46.  35.  31.  98.  27. 121. 102.  49. 120.  72.  16.
  39.  10.  65.   5.  14. 106.  19.  60. 109.   9.   6. 100. 102. 114.
  85.   4. 114.   1.]
Epoch = 400 loss=0.011931799352169037 train_acc = 0.987 te

KeyboardInterrupt: 

### Adding Convolutions

In [200]:
data = gen_data(N=10000)
features = data[0]
print("Feature Vectors:")
print(features)
labels = data[1]
print("Labels:")
print(labels)
test_data = gen_data(N=2000)
features_test = test_data[0]
print("Test Feature Vectors:")
print(features_test)
labels_test = test_data[1]
print("Test Labels:")
print(labels_test)

Feature Vectors:
[[ 39  55   4 ...   1  74  39]
 [ 27  69 117 ... 123 114 115]
 [ 33  62   8 ...  88 115  46]
 ...
 [ 21  36   9 ...  38  52  80]
 [120  21  78 ...  62 120 118]
 [ 81  75  55 ...   7   3  18]]
Labels:
[[ 0 55  0 ...  0  0  0]
 [ 0 69  0 ...  0  0  0]
 [ 0 62  0 ...  0  0  0]
 ...
 [ 0 36  0 ...  0  0  0]
 [ 0 21  0 ...  0  0  0]
 [ 0 75  0 ...  0  0  0]]
Test Feature Vectors:
[[ 44  53   0 ...   6  63  41]
 [100 118  50 ...  85 110 125]
 [124  14   9 ...  44 124  26]
 ...
 [122  33   0 ...  10  18  37]
 [121  36 121 ...  81  47  64]
 [ 76 118   6 ... 123  44  74]]
Test Labels:
[[  0  53   0 ...   0   0   0]
 [  0 118   0 ...   0   0   0]
 [  0  14   0 ...   0   0   0]
 ...
 [  0  33   0 ...   0   0   0]
 [  0  36   0 ...   0   0   0]
 [  0 118   0 ...   0   0   0]]


In [202]:
net = nn.Sequential(nn.Conv2d(1, 8, kernel_size=(1,3), stride=1),
                    nn.ReLU(),
                    nn.Conv2d(8, 16, kernel_size=(1,3), stride=1),
                    nn.ReLU(),
                    nn.Conv2d(16, 32, kernel_size=(1,3), stride=1),
                    nn.ReLU(),
                    nn.Flatten(start_dim=1, end_dim=-1),
                   )
net(torch.from_numpy(features[0]).unsqueeze(0).unsqueeze(0).unsqueeze(0).float()).shape 

torch.Size([1, 832])

In [184]:
net = nn.Sequential(nn.Conv2d(1, 8, kernel_size=(1,3), stride=1),
                    nn.ReLU(),
                    nn.Conv2d(8, 16, kernel_size=(1,3), stride=1),
                    nn.ReLU(),
                    nn.Conv2d(16, 32, kernel_size=(1,3), stride=1),
                    nn.ReLU(),
                    nn.Flatten(start_dim=1, end_dim=-1),
                    nn.Linear(832, 32)
                   )

In [185]:
#criterion = nn.CrossEntropyLoss()
criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=1e-3)

In [25]:
def validate(net, test_data, test_target, net_type=None):
    net = net.eval()
    
    with torch.no_grad():
        test_data_torch = torch.from_numpy(test_data).float().to(device)
        test_target_torch = torch.from_numpy(test_target).float().to(device)

        if net_type=='conv':
            test_data_torch = test_data_torch.unsqueeze(1).unsqueeze(2)
        
        test_pred = np.rint(net(test_data_torch).detach().numpy())
        
        i = random.randint(0, test_data.shape[0] - 1)
        print("Pred[", i, "]:")
        print(test_pred[i])
        print("Target[", i, "]:")
        print(test_target_torch.detach().numpy()[i])
        print("Input[", i, "]:")
        print(test_data_torch.detach().numpy()[i])

        test_accuracy = (test_target_torch.detach().numpy() == test_pred).mean()
        test_rmse = (np.sqrt((test_pred-test_target_torch.detach().numpy())**2).mean())

    return test_accuracy, test_rmse


In [187]:
net = train_net(net, features, labels, features_test, labels_test, N_epochs=5000, print_freq=10, net_type='conv')

torch.save(net, "conv_0")

train_data_torch
tensor([[ 94.,  68.,   8.,  ...,  74.,  13.,  60.],
        [  4.,  33.,  70.,  ...,  77., 117.,  48.],
        [  8.,  50.,  52.,  ...,  89.,  98.,  91.],
        ...,
        [ 77.,  96., 106.,  ...,  93.,  28.,  67.],
        [ 72., 106., 114.,  ...,  22.,  49.,   1.],
        [ 27.,  79.,   6.,  ...,  13.,  13.,  77.]])
train_target_torch
tensor([[  0.,  68.,   0.,  ...,   0.,   0.,   0.],
        [  0.,  33.,   0.,  ...,   0.,   0.,   0.],
        [  0.,  50.,   0.,  ...,   0.,   0.,   0.],
        ...,
        [  0.,  96.,   0.,  ...,   0.,   0.,   0.],
        [  0., 106.,   0.,  ...,   0.,   0.,   0.],
        [  0.,  79.,   0.,  ...,   0.,   0.,   0.]])
Pred[ 918 ]:
[ 2.  0.  1. -0.  0.  5. -2. -2.  1. -4. -1. -1. -2.  1.  2.  2.  1.  3.
 -1.  1.  0.  1.  2. -6. -4.  1.  4. -3. -0. -1. -1.  0.]
Target[ 918 ]:
[ 0. 63.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Input[ 918 ]:
[[[ 38. 

Pred[ 958 ]:
[ 0. 59. -0.  0.  0. -0. -0. -0.  1.  0.  0. -0.  0.  1.  1.  0.  1.  0.
 -0. -0. -1.  0.  0.  0.  1. -0. -0.  0.  0.  0.  0.  0.]
Target[ 958 ]:
[ 0. 80.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Input[ 958 ]:
[[[ 81.  80.  22.  27.  61.  46.  64.  16.  50.  10.  77.  14.  90.  25.
    68.  64.  55.  42.  62. 118.  12.  97.  32. 100.  32.  67. 107.  88.
    27.  91.  74.  72.]]]
Pred[ 522 ]:
[-0. 61. -0.  0. -0. -0.  0.  1.  1. -0. -0.  0. -0. -0. -0.  0.  0. -0.
 -0.  0.  0.  0.  0.  0.  1. -0.  1. -1.  1. -1.  0.  0.]
Target[ 522 ]:
[  0. 122.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.]
Input[ 522 ]:
[[[ 33. 122.  73.  82.  32.  84.  21.  14.  74.  58.  95.  32.  83.  61.
    78.  13.  20.  89.  11.  62.  51.   8.  87.  68.  14.  62.  99.  96.
    70.  40.  37.  75.]]]
Epoch = 80 loss

Pred[ 2165 ]:
[-1. 56. -2.  1. -1.  0. -1.  1.  0.  1. -0.  1. -1.  0.  1.  0. -2.  0.
 -1.  1. -0.  1. -1.  0. -0.  1. -1. -0.  0.  1.  0.  1.]
Target[ 2165 ]:
[ 0. 62.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Input[ 2165 ]:
[[[ 14.  62.  70.  70.  22.   6.  96.  32. 123.  99.  80.  67.  16.  24.
    98.  92.  54.  77.  91.  29.  35.  47. 103.  43.  38.  84.   1.  14.
    39.  19.  97. 105.]]]
Pred[ 1655 ]:
[-0. 59. -1.  0. -0. -1. -1. -0. -1.  1.  0.  1. -1. -0.  1. -0. -1. -0.
 -1.  2. -1.  0. -0. -0. -0.  2. -1. -0. -0.  1.  1.  0.]
Target[ 1655 ]:
[ 0. 57.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Input[ 1655 ]:
[[[ 94.  57.  65.   2. 105.   3.   7.  68.  99.  57.  10.  55.  59.  84.
    91. 119.   2.  19.  31. 115.  43.  42.  61. 118.  54.  60.  82.  53.
   114.  76.  76.  47.]]]
Epoch = 160 loss=3.0030295848846436 train_

Pred[ 7277 ]:
[-0. 50.  0. -0.  0. -0.  1.  0.  0. -0.  0.  0. -1.  0. -0.  0.  0. -0.
  1. -1.  0. -0.  0. -0.  0.  0.  0. -0. -0.  0.  1. -1.]
Target[ 7277 ]:
[ 0. 48.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Input[ 7277 ]:
[[[ 41.  48.  79.  63.  31.  37.  98. 108.  19.  92.  51.  34.  72. 105.
    38.  23.  80.  48.  54.  81.  81. 114.  88.  18.  67.  38. 102.  65.
    75.   5.  57. 100.]]]
Pred[ 1374 ]:
[  1. 103.  -0.   0.   0.  -0.   1.   0.   0.  -1.   0.   1.  -0.   0.
   0.  -0.   1.  -0.   1.  -1.   0.  -1.   1.  -1.   0.   1.   1.  -0.
  -0.  -0.   0.  -1.]
Target[ 1374 ]:
[  0. 102.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.]
Input[ 1374 ]:
[[[104. 102.   9. 101.  37.  45.  60.  83.  81. 119.  13. 107.   1.  10.
    65. 103.  70.  14.  85.  99.  18.  60.  97.  20.  42.  22. 112.  24.
  

Pred[ 8526 ]:
[ 0. 53.  0.  0.  0. -0.  0. -0.  0. -0.  0. -1. -0. -0. -0. -1.  1. -1.
 -1.  0. -1.  0.  1. -0. -0.  0.  0. -0.  0.  0.  0.  0.]
Target[ 8526 ]:
[ 0. 53.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Input[ 8526 ]:
[[[ 74.  53.  99. 112.  87.  16. 116.  48.  30.  15.   8. 125.  80.  70.
    64.  15.  14.  26. 125.  23.   8.  81. 113.  24.  61.  87.  24.  97.
    38.  28.  97.  90.]]]
Pred[ 1149 ]:
[ 1. 34.  1.  1. -0.  0. -0. -0.  1. -0. -0.  0.  0.  0.  0. -0.  1. -0.
  0. -0. -1.  2.  0.  0.  0.  0.  1. -0.  0.  1. -0. -0.]
Target[ 1149 ]:
[ 0. 35.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Input[ 1149 ]:
[[[  1.  35. 108. 110.   7.  85.  64. 110.  76.  84.  36.  42.  15.  20.
   117.  56. 102.  29.   2.  27.  97.  63.  29.  96.  49.  66.  31.  33.
    78.  93.   1.   9.]]]
Epoch = 320 loss=0.2663145661354065 train_

...

Pred[ 6223 ]:
[-0. 79.  0. -0.  0. -0.  0. -0. -0.  0. -0.  0. -0. -0.  0.  0.  0. -0.
 -0. -0. -0. -0. -0. -0.  0. -0. -0.  0. -0. -0.  0. -0.]
Target[ 6223 ]:
[ 0. 79.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Input[ 6223 ]:
[[[  2.  79.  74. 106.  82.  69. 125.  21. 117.  44.  60.  89.  48. 116.
    33.  81.  56.  83.  45. 124. 115. 114. 126. 120.  52. 109. 119.  40.
    40.  72.  27.  44.]]]
Pred[ 1445 ]:
[ 0.  4.  0.  0. -0.  0. -0. -0. -0. -0. -0. -0. -0.  0.  0. -0.  0.  0.
 -0.  0. -0.  0.  0. -0.  0. -0.  0.  0. -0.  0. -0. -0.]
Target[ 1445 ]:
[0. 4. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]
Input[ 1445 ]:
[[[ 40.   4.  88.  21.  36. 105.   8.  84.  58. 117.  12.  71.  34.  39.
    29. 118. 126.  31.  60. 108.  45.  37.  80. 105.  57.  77.  77.  24.
   108.  80.  82.   3.]]]
Epoch = 4640 loss=0.005536056589335203 train_acc = 0.998 test_acc = 0.999 

Pred[ 2520 ]:
[  0. 108.   0.   0.   0.  -0.  -0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.  -0.   0.  -0.  -0.  -0.   0.   0.   0.  -0.  -0.   0.   0.
   0.  -0.   0.  -0.]
Target[ 2520 ]:
[  0. 108.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.]
Input[ 2520 ]:
[[[ 75. 108.  91.   8. 100.  69. 116.  38. 109.  37.  22.  38.  85.  97.
    38.  62. 117.  39.  31.  33.  54. 118.  19.  82.  92.   5.  12.  32.
    70.  40.   9.  10.]]]
Pred[ 409 ]:
[ 0. 31.  0.  0.  0. -0. -0.  0.  0.  0.  0.  0.  0. -0.  0.  0. -0. -0.
 -0. -0. -0.  0.  0.  0.  0. -0.  0.  0.  0.  0.  0. -0.]
Target[ 409 ]:
[ 0. 31.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Input[ 409 ]:
[[[ 95.  31.  56.  22.  83.  57.  81.  76.  76.  97.  55.  27.  42.  21.
    23. 105.  13.  61.  88.   4. 111. 114.  75. 112.  82.  70.  59.  75.
    1

Pred[ 1016 ]:
[ 0. 92. -0. -0.  0. -0.  0. -0. -0. -0.  0. -0.  0. -0. -0.  0.  0. -0.
 -0.  0. -0. -0. -0.  0. -0. -0.  0.  0. -0.  0.  0. -0.]
Target[ 1016 ]:
[ 0. 92.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Input[ 1016 ]:
[[[  8.  92.  95.  31.  84. 124.  88.  81.  31.  83.  52.  59.  24.  53.
     1. 126.   1.  40.  16.  21. 115. 116. 116.  98.  51. 107.  22.  82.
     3.  88. 107.  92.]]]
Pred[ 1060 ]:
[-0. 72.  0.  0.  0.  0.  0.  0. -0.  0. -0. -0.  0. -0. -0.  0. -0. -0.
 -0. -0. -0. -0. -0.  0. -0. -0.  0.  0.  0.  0.  0. -0.]
Target[ 1060 ]:
[ 0. 72.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Input[ 1060 ]:
[[[ 21.  72.  28.  83. 124.  39.  66.  63.  32.   2.   5.  82.  11. 107.
    58.  17.  55.  74.  23. 124. 112.  96.  34.  87.  58.  22. 102.  67.
    66.  51.  55.  53.]]]
Epoch = 4800 loss=0.013146325945854187 tra

Pred[ 9825 ]:
[ 0. 70.  0.  0. -0. -0. -0.  0.  0. -0.  0. -0.  0. -0. -0. -0. -0. -0.
  0. -0.  0. -0.  0. -0. -0.  0.  0.  0.  0.  0.  0.  0.]
Target[ 9825 ]:
[ 0. 70.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Input[ 9825 ]:
[[[ 89.  70.  22.  33.  26.  13.  68.  28.  49.  39.  35.  55.   8.  88.
   105. 120.  63.  55.  22.  28.  50.   9. 124.  96.  91.  66.  54.  42.
    40.   4.   7.  92.]]]
Pred[ 1306 ]:
[ 0.  6. -0.  0.  0. -0. -0.  0. -0. -0.  0. -0.  0. -0. -0. -0.  0. -0.
  0. -0.  0. -0. -0. -0. -0.  0.  0.  0.  0. -0.  0. -0.]
Target[ 1306 ]:
[0. 6. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]
Input[ 1306 ]:
[[[  7.   6.  32. 103. 117.  70.  75.  19.  45. 116.  75.  78.  48.  66.
   108.  21. 114.  93.  87.  58.  29.  49.  11.   0.  75.  68.  84.  56.
    92.  43.  40.   1.]]]
Epoch = 4880 loss=0.0208786278963089 train_acc = 0.998 test_acc = 0.998 tr

Pred[ 4329 ]:
[-0. 12.  0.  0.  0.  0.  0. -0.  0. -0.  0.  0. -0.  0. -0.  0.  0. -0.
 -0.  0. -0. -0. -0. -0. -0.  0. -0. -1.  0.  0.  0.  0.]
Target[ 4329 ]:
[ 0. 13.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Input[ 4329 ]:
[[[107.  13.  21.  58.  35.   2. 118.  76.  55. 119.  54.   1.  84.  78.
    37.  13. 104.  85. 113.  31.  35.  30.  77.  45.  75.  83.  61. 118.
    48.  34. 101.  76.]]]
Pred[ 1112 ]:
[-0. 60.  0. -0.  1.  1.  0.  0.  0. -0.  0. -0. -0.  0. -0.  0.  0. -0.
 -0.  0. -0. -0. -0. -0. -0.  0.  0. -1.  0.  0.  0.  0.]
Target[ 1112 ]:
[ 0. 61.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Input[ 1112 ]:
[[[ 32.  61.  77. 117.  76.  49.  56.  27.  55.  86.  56.  87.  38.  87.
     9.  68. 115.  95. 103.  46.  83. 113.  78. 123.  50. 119. 125. 109.
    42. 117.   1. 104.]]]
Epoch = 4960 loss=0.08951538801193237 trai

### Adding Mask to the Input Data

In [33]:
# generate N vectors of 32 bytes
# generate N label vectors of 31 zero bytes and 1 non-zero byte at target_idx
def gen_data(N=5000, d=32, low=0, high=127, target_idx=1):
    data = np.random.randint(low=low, high=high, size=(N,d))
    labels = np.zeros((N, d), dtype=int)
    labels[:, target_idx] = data[:, target_idx]
    return data, labels

In [34]:
target_idx = 1
N = 5000
d = 32

data = gen_data(N=N)

features_0 = data[0]

features_1 = np.zeros((N, d), dtype=int)
features_1[:, target_idx] = 1

features = np.array([features_0, features_1])
print("Feature Vectors:")
print(features)

labels = data[1]
print("Labels:")
print(labels)

N=1000
test_data = gen_data(N=N)

features_test_0 = test_data[0]

features_test_1 = np.zeros((N, d), dtype=int)
features_test_1[:, target_idx] = 1

features_test = np.array([features_test_0, features_test_1])

print("Test Feature Vectors:")
print(features_test)
labels_test = test_data[1]
print("Test Labels:")
print(labels_test)

Feature Vectors:
[[[ 41 116 118 ...  39  76  29]
  [ 69  66  97 ...  74  97 102]
  [104  98  70 ...  94  96 117]
  ...
  [ 33  12   9 ...  31  38 126]
  [104  23  63 ...  51  28  72]
  [ 39 105  88 ...  88  34   9]]

 [[  0   1   0 ...   0   0   0]
  [  0   1   0 ...   0   0   0]
  [  0   1   0 ...   0   0   0]
  ...
  [  0   1   0 ...   0   0   0]
  [  0   1   0 ...   0   0   0]
  [  0   1   0 ...   0   0   0]]]
Labels:
[[  0 116   0 ...   0   0   0]
 [  0  66   0 ...   0   0   0]
 [  0  98   0 ...   0   0   0]
 ...
 [  0  12   0 ...   0   0   0]
 [  0  23   0 ...   0   0   0]
 [  0 105   0 ...   0   0   0]]
Test Feature Vectors:
[[[ 91  92  57 ... 108  74   3]
  [ 63 124  38 ...  56  93  90]
  [ 56  69 106 ... 111  26  94]
  ...
  [105  96 113 ...  67 115  68]
  [ 43  68  34 ...  39  20  11]
  [ 59  14  51 ...  69 108  72]]

 [[  0   1   0 ...   0   0   0]
  [  0   1   0 ...   0   0   0]
  [  0   1   0 ...   0   0   0]
  ...
  [  0   1   0 ...   0   0   0]
  [  0   1   0 ...   0   0 

In [35]:
net = nn.Sequential(nn.Conv2d(2, 8, kernel_size=(1,3), stride=1),
                    nn.ReLU(),
                    nn.Conv2d(8, 16, kernel_size=(1,3), stride=1),
                    nn.ReLU(),
                    nn.Conv2d(16, 32, kernel_size=(1,3), stride=1),
                    nn.ReLU(),
                    nn.Flatten(start_dim=1, end_dim=-1),
                   )

print(torch.from_numpy(features).unsqueeze(0).float())


net(torch.from_numpy(features).unsqueeze(0).float()).shape 

tensor([[[[ 41., 116., 118.,  ...,  39.,  76.,  29.],
          [ 69.,  66.,  97.,  ...,  74.,  97., 102.],
          [104.,  98.,  70.,  ...,  94.,  96., 117.],
          ...,
          [ 33.,  12.,   9.,  ...,  31.,  38., 126.],
          [104.,  23.,  63.,  ...,  51.,  28.,  72.],
          [ 39., 105.,  88.,  ...,  88.,  34.,   9.]],

         [[  0.,   1.,   0.,  ...,   0.,   0.,   0.],
          [  0.,   1.,   0.,  ...,   0.,   0.,   0.],
          [  0.,   1.,   0.,  ...,   0.,   0.,   0.],
          ...,
          [  0.,   1.,   0.,  ...,   0.,   0.,   0.],
          [  0.,   1.,   0.,  ...,   0.,   0.,   0.],
          [  0.,   1.,   0.,  ...,   0.,   0.,   0.]]]])


torch.Size([1, 4160000])

In [44]:
net = nn.Sequential(nn.Conv2d(2, 8, kernel_size=(1,3), stride=1),
                    nn.ReLU(),
                    nn.Conv2d(8, 16, kernel_size=(1,3), stride=1),
                    nn.ReLU(),
                    nn.Conv2d(16, 32, kernel_size=(1,3), stride=1),
                    nn.ReLU(),
                    nn.Flatten(start_dim=1, end_dim=-1),
                    nn.Linear(4160000, 32)
                   )

def train_net(net, train_data, train_target, test_data, test_target, N_epochs=None, print_freq=100, net_type=None):

#     print("Train Vectors:")
#     print(train_data)
#     print("Train Labels:")
#     print(train_target)
#     print("Test Vectors:")
#     print(test_data)
#     print("Test Labels:")
#     print(test_target)
    
    net = net.to(device)
    train_data_torch = torch.from_numpy(train_data).float().to(device)
    train_target_torch = torch.from_numpy(train_target).float().to(device)
    
    print("train_data_torch")
    print(train_data_torch)
    print("train_target_torch")
    print(train_target_torch)
    
    if net_type=='conv':
        train_data_torch = train_data_torch.unsqueeze(0)

    net = net.train()
    for i in range(N_epochs):
        #pred = net(train_data_torch.float())
        #loss = criterion(pred, train_target_torch.float())
        pred = net(train_data_torch)
        loss = criterion(pred, train_target_torch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % print_freq == 0:
            train_accuracy, train_rmse = validate(net, train_data, train_target, net_type=net_type)
            test_accuracy, test_rmse = validate(net, test_data, test_target, net_type=net_type)
            net = net.train()
            print(f'Epoch = {i} loss={loss} train_acc = {train_accuracy:.3f} test_acc = {test_accuracy:.3f} train_rmse = {train_rmse:.3f}  test_rmse = {test_rmse:.3f}')
            
    return net


def validate(net, test_data, test_target, net_type=None):
    print("here!")
    net = net.eval()
    
    with torch.no_grad():
        print(torch.from_numpy(test_data))
        test_data_torch = torch.from_numpy(test_data).float().to(device)
        test_target_torch = torch.from_numpy(test_target).float().to(device)

        if net_type=='conv':
            test_data_torch = test_data_torch.unsqueeze(0)
        
        print(net(test_data_torch))
        test_pred = np.rint(net(test_data_torch).detach().numpy())
        print("here pred!")
        print(test_pred)
        
        #print(test_pred)
        
        #i = random.randint(0, test_data[0].shape[0] - 1)
        #print("Pred[", i, "]:")
        #print(test_pred[0][i])
        #print("Target[", i, "]:")
        #print(test_target_torch.detach().numpy()[i])
        #print("Input[", i, "]:")
        #print(test_data_torch.detach().numpy()[i])

        test_accuracy = (test_target_torch.detach().numpy() == test_pred).mean()
        test_rmse = (np.sqrt((test_pred-test_target_torch.detach().numpy())**2).mean())

    return test_accuracy, test_rmse


In [45]:
criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=1e-3)

net = train_net(net, features, labels, features_test, labels_test, N_epochs=5000, print_freq=10, net_type='conv')

torch.save(net, "conv_1")

train_data_torch
tensor([[[ 41., 116., 118.,  ...,  39.,  76.,  29.],
         [ 69.,  66.,  97.,  ...,  74.,  97., 102.],
         [104.,  98.,  70.,  ...,  94.,  96., 117.],
         ...,
         [ 33.,  12.,   9.,  ...,  31.,  38., 126.],
         [104.,  23.,  63.,  ...,  51.,  28.,  72.],
         [ 39., 105.,  88.,  ...,  88.,  34.,   9.]],

        [[  0.,   1.,   0.,  ...,   0.,   0.,   0.],
         [  0.,   1.,   0.,  ...,   0.,   0.,   0.],
         [  0.,   1.,   0.,  ...,   0.,   0.,   0.],
         ...,
         [  0.,   1.,   0.,  ...,   0.,   0.,   0.],
         [  0.,   1.,   0.,  ...,   0.,   0.,   0.],
         [  0.,   1.,   0.,  ...,   0.,   0.,   0.]]])
train_target_torch
tensor([[  0., 116.,   0.,  ...,   0.,   0.,   0.],
        [  0.,  66.,   0.,  ...,   0.,   0.,   0.],
        [  0.,  98.,   0.,  ...,   0.,   0.,   0.],
        ...,
        [  0.,  12.,   0.,  ...,   0.,   0.,   0.],
        [  0.,  23.,   0.,  ...,   0.,   0.,   0.],
        [  0., 105.,   

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x832000 and 4160000x32)